# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## Python 中的多線程

In [1]:
import _thread
import time

def print_time( threadName, data, delay):
    for d in data:
        time.sleep(delay)
        print(threadName, ' => ', d)

_thread.start_new_thread( print_time, ("Thread-1", range(0, 5, 2), 2, ) )
_thread.start_new_thread( print_time, ("Thread-2", range(1, 5, 2), 5, ) )


123145491259392

## 多線程爬蟲


In [2]:
import requests
import time

urls = ['http://python.org', 'http://python.org', 'http://python.org',
        'http://python.org', 'http://python.org']

startTime = time.time()

for url in urls:
    requests.get(url)

finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間

Thread-1  =>  0
1.1034669876098633
Thread-1  =>  2
Thread-2  =>  1
Thread-1  =>  4
Thread-2  =>  3


In [3]:
import requests
import _thread
import time

urls = ['http://python.org', 'http://python.org', 'http://python.org',
        'http://python.org', 'http://python.org']

startTime = time.time()

for url in urls:
    _thread.start_new_thread( requests.get, (url, ) )

finishTime = time.time()
print(finishTime - startTime) # 利用 thread 的爬蟲所需時間

0.00023102760314941406


## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [4]:
import requests
from bs4 import BeautifulSoup
import re
import os
import csv
import time
import random
import pandas as pd

In [5]:
url='https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_National_Pokédex_number'
r=requests.get(url)
r.encoding = 'utf-8'
soup=BeautifulSoup(r.text,"html5lib")

In [6]:
link_list=[]
for i in soup.find('div',{'id':'mw-content-text'}).find_all('tr',{"style":"background:#FFF"})[:50]:

    #link to specific attr
    link='https://bulbapedia.bulbagarden.net/'+i.find_all('a',
                                                          href=re.compile("^(/wiki/)((?!;)\S)*$"))[1]['href']
    #print('Link: ',link)   
    link_list.append(link)
    
    #get specific attr for a pokemon

In [18]:
n=0
def parse_t(link):
    url_pkm=link
    r_pkm=requests.get(url_pkm)
    r_pkm.encoding = 'utf-8'
    soup_pkm=BeautifulSoup(r_pkm.text,"html5lib")
    pokemon_data=[]
    try:
        #hp
        hp=soup_pkm.find('tr',{'style':"background: #FF5959; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("HP: ",hp) 
        pokemon_data.append(hp)
        #atk
        atk=soup_pkm.find('tr',{'style':"background: #F5AC78; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("ATK: ",atk) 
        pokemon_data.append(atk)
        #def
        defense=soup_pkm.find('tr',{'style':"background: #FAE078; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("DEF: ",defense)
        pokemon_data.append(defense)
        #sp_atk
        sp_atk=soup_pkm.find('tr',{'style':"background: #9DB7F5; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SP_ATK: ",sp_atk)  
        pokemon_data.append(sp_atk)
        #sp_def
        sp_defense=soup_pkm.find('tr',{'style':"background: #A7DB8D; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SP_DEF: ",sp_defense)
        pokemon_data.append(sp_defense)
        #spd
        spd=soup_pkm.find('tr',{'style':"background: #FA92B2; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SPD: ",spd)
        pokemon_data.append(spd)

        
        #type_effectiveness
        type_effectiveness=soup_pkm.find('table',{'class':'roundy','width':'100%','align':'center'}) 
        #get four dimentions 
        normal=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[1]
        weak=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[2]
        immue=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[3]
        resisant=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[4]
        #create list for each dimention
        normal_lst=[]
        weak_lst=[]
        immue_lst=[]
        resisant_lst=[]
    
    
        #get data into each list and write
        #Damaged Normally by
        for j in normal.find_all('span',{'style':'display:inline-block;'}):
            normal_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Damaged Normally by :',normal_lst)
        pokemon_data.append(str(normal_lst))
    
    
        #Weak to
        for j in weak.find_all('span',{'style':'display:inline-block;'}):
            weak_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Weak to :',weak_lst)
        pokemon_data.append(str(weak_lst))
    
    
        #Immue to
        for j in immue.find_all('span',{'style':'display:inline-block;'}):
            immue_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Immue to :',immue_lst)
        pokemon_data.append(str(immue_lst))
    
    
        #Resisant to
        for j in resisant.find_all('span',{'style':'display:inline-block;'}):
            resisant_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Resisant to :',resisant_lst)
        pokemon_data.append(str(resisant_lst))
        print('Done!!!')
    
        with open('/users/mac/0517.csv', 'a', newline='') as file:
            writer = csv.writer(file, delimiter=',')
            global n
            n=n+1
            writer.writerow(pokemon_data)
    
    except:
        pass


In [19]:
n=0
def test():
    with open('/users/mac/0517.csv', 'a', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        global n
        n=n+1
        writer.writerow([n]) 
for i in range(10):
    _thread.start_new_thread( test,() )

# 多線所需時間

In [26]:
startTime=time.time()
for i in link_list[:]:
    _thread.start_new_thread( parse_t,(i,) )

finishTime = time.time()
print(finishTime - startTime) # 

0.0028018951416015625


# 正常所需時間

In [25]:
#arrange data
counter=0 #counter
startTime = time.time()
n=0
for i in link_list:
    
    #get specific attr for a pokemon
    url_pkm=i
    r_pkm=requests.get(url_pkm)
    r_pkm.encoding = 'utf-8'
    soup_pkm=BeautifulSoup(r_pkm.text,"html5lib")

    try:
        #hp
        hp=soup_pkm.find('tr',{'style':"background: #FF5959; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("HP: ",hp) 
        pokemon_data.append(hp)
        #atk
        atk=soup_pkm.find('tr',{'style':"background: #F5AC78; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("ATK: ",atk) 
        pokemon_data.append(atk)
        #def
        defense=soup_pkm.find('tr',{'style':"background: #FAE078; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("DEF: ",defense)
        pokemon_data.append(defense)
        #sp_atk
        sp_atk=soup_pkm.find('tr',{'style':"background: #9DB7F5; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SP_ATK: ",sp_atk)  
        pokemon_data.append(sp_atk)
        #sp_def
        sp_defense=soup_pkm.find('tr',{'style':"background: #A7DB8D; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SP_DEF: ",sp_defense)
        pokemon_data.append(sp_defense)
        #spd
        spd=soup_pkm.find('tr',{'style':"background: #FA92B2; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SPD: ",spd)
        pokemon_data.append(spd)

        
        #type_effectiveness
        type_effectiveness=soup_pkm.find('table',{'class':'roundy','width':'100%','align':'center'}) 
        #get four dimentions 
        normal=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[1]
        weak=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[2]
        immue=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[3]
        resisant=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[4]
        #create list for each dimention
        normal_lst=[]
        weak_lst=[]
        immue_lst=[]
        resisant_lst=[]
    
    
        #get data into each list and write
        #Damaged Normally by
        for j in normal.find_all('span',{'style':'display:inline-block;'}):
            normal_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Damaged Normally by :',normal_lst)
        pokemon_data.append(str(normal_lst))
    
    
        #Weak to
        for j in weak.find_all('span',{'style':'display:inline-block;'}):
            weak_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Weak to :',weak_lst)
        pokemon_data.append(str(weak_lst))
    
    
        #Immue to
        for j in immue.find_all('span',{'style':'display:inline-block;'}):
            immue_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Immue to :',immue_lst)
        pokemon_data.append(str(immue_lst))
    
    
        #Resisant to
        for j in resisant.find_all('span',{'style':'display:inline-block;'}):
            resisant_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Resisant to :',resisant_lst)
        pokemon_data.append(str(resisant_lst))
    
    
        n=n+1
        print(n)
    except:
        continue

print('Done!!!')
finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間

Done!!!
68.32529616355896
